In [1]:
import pandas as pd
import scipy.stats
import numpy as np
import os

In [2]:
data = os.listdir("processed data")

In [3]:
ns_files = [a for a in data if "ns" in a ]

In [51]:
past_files = [a for a in data if ("ns" not in a and "line" not in a and "rookie" not in a)]

In [5]:
df = pd.read_csv("processed data/"+ns_files[0])

In [14]:
len(rep_values)

29

In [25]:
#len(rookie_infos.values[0][:5].tolist() + rep_values)

In [53]:
for ns, ps in zip(ns_files, past_files):
    
    df = pd.read_csv('processed data/'+ns)
    pdf = pd.read_csv("processed data/"+ps)
    rep_df = df.copy()
    
    pos = ns[:2]
    
    infos = df[df["YDS_RUSHING/G_avg"].isna()][["pid", "AVG"]].values
    
    reps = []
    for info in infos:
        rookie_adp = info[1]
        rookie_pid = info[0]

        rookie_infos = rep_df[rep_df.pid == rookie_pid]
        nearest_values = []
        for i in pdf.Season.value_counts().index.values:
            sdf = pdf[pdf.Season == i]

            adps = sdf.AVG.values

            closest_number = min(adps, key = lambda x: abs( rookie_adp - x))

            closest_player = sdf[sdf.AVG == closest_number].values


            nearest_values.append(closest_player[0][5:-2].tolist())

        rep_values = pd.DataFrame(nearest_values, columns = pdf.columns[5:-2]).mean(numeric_only = True).values.tolist()#.mean(axis=1)

        bdf = rookie_infos.values[0][:5].tolist() + rep_values + rookie_infos.values[0][-3:].tolist()
        reps.append(pd.DataFrame([bdf], columns=df.columns))

    reps_full = pd.concat(reps)
    new_df = pd.concat([rep_df, reps_full])
    new_df = new_df.drop_duplicates(["pid"], keep="last")

    new_df = new_df.sort_values("AVG")
    
    new_df.to_csv(f"processed data/{pos}_rookie.csv", index=False)